# 🧬 Podręcznik użytkownika: Quantum Protein Folding

Witamy w kompleksowym przewodniku po projekcie **Quantum Protein Folding**.

Nasza implementacja jest **zainspirowana i rozszerza** metody opisane w pracy 
[*Protein Folding Problem: A Quantum Approach*](https://arxiv.org/pdf/1908.02163) oraz w tym repozytorium: [quantum-protein-folding-qiskit](https://github.com/qiskit-community/quantum-protein-folding)


### 🎯 Cele
Po przerobieniu tego notatnika zrozumiesz:
1. **Abstrakcję**: W jaki sposób mapujemy problem biologiczny na podejście obliczeń kwantowych.
2. **Hamiltoniany**: Jak penalizujemy "niemożliwe" skręty (np. rozerwanie łańcucha) i nagradzamy "dobre" (kontakty hydrofobowe).
3. **Algorytm**: W jaki sposób VQE (Variational Quantum Eigensolver) znajduje optymalną strukturę.

### ⚠️ Ograniczenia i limity
> **Ograniczenia długości łańcucha:**
> * **Minimalna długość:** Symulacja wymaga łańcucha o długości **N ≥ 5**. Jest to wymuszane przez stałą `MIN_CHAIN_LENGTH`.
> * **Maksymalna długość:** Chociaż baza kodu teoretycznie obsługuje nieograniczone długości łańcuchów, zalecamy wybór łańcuchów do **N ≈ 9-12**, ponieważ wymagane zasoby obliczeniowe skalują się wykładniczo.
> * **Obsługiwane symbole aminokwasów:** Obsługujemy podstawowe `20` jednoliterowych symboli aminokwasów, ale ten model umożliwia wprowadzenie własnych symboli i odpowiadających im energii par, modyfikując plik `mj_matrix.txt` w katalogu `resources`.
>   
> *Rekomendacja:* W tym demo trzymamy się **N=7 (APRLRFY)**, aby zapewnić szybkie wykonanie i odwzorować wyniki przedstawione w [rozwiązaniu referencyjnym](https://github.com/qiskit-community/quantum-protein-folding).

Aby uzyskać więcej informacji na temat założeń i konfiguracji stałych, odwiedź [oficjalną dokumentację](https://qfold-thesis.github.io/quantum-protein-folding/).

## ⚙️ Krok 0: Konfiguracja środowiska

Zanim przejdziesz dalej, upewnij się, że wykonałeś kroki instalacyjne wymagane w pliku [`README`](https://github.com/QFold-Thesis/quantum-protein-folding?tab=readme-ov-file#-installation) projektu.

Ponieważ ten notatnik znajduje się w katalogu `docs/`, musimy ręcznie wskazać Pythonowi lokalizację kodu źródłowego (`src/`).

In [9]:
import sys
import os
from pathlib import Path

# Znajdź katalog główny projektu względem tego notatnika
current_dir = Path(os.getcwd())
project_src = current_dir.parent / "src"

# Dodaj do sys.path
if str(project_src) not in sys.path:
    sys.path.append(str(project_src))
    print("✅ Ścieżka systemowa zaktualizowana")

# Import narzędzi projektu
import logging
from logger import get_logger
from IPython.display import Image, display

logger = get_logger()
logger.disabled = True

---
# 🟢 Część 1: Zautomatyzowany przepływ pracy

Dla większości użytkowników moduł `setup_utils` zapewnia doświadczenie typu "Battery Included" (gotowe do użycia). Zajmuje się on orkiestracją klas za Ciebie.

### 🧪 1. Zdefiniuj sekwencję białka

Definiujemy białko jako ciąg znaków. Poprawność tych znaków zależy od używanego **Modelu Oddziaływań** (zdefiniowanego w `constants.py`).

| Model | Opis |
| :--- | :--- |
| **MJ** | Wykorzystuje realistyczne energie kontaktowe Miyazawa-Jernigan. |
| **HP** | Model uproszczony. Węzły hydrofobowe przyciągają się nawzajem, polarne są neutralne. |

**Pro-Tip:** Jeśli zobaczysz błąd `UnsupportedAminoAcidSymbolError`, sprawdź, czy Twoja sekwencja pasuje do danych wybranego modelu.

In [10]:
from constants import EMPTY_SIDECHAIN_PLACEHOLDER
from utils.setup_utils import setup_folding_system

# Zdefiniuj swoją sekwencję tutaj
main_chain = "APRLRFY"
side_chain = EMPTY_SIDECHAIN_PLACEHOLDER * len(main_chain)

print(f"🧬 Analiza sekwencji: {main_chain}")

# Funkcja konfigurująca automatycznie:
# 1. Tworzy instancję obiektu Protein
# 2. Wybiera model oddziaływań (MJ/HP)
# 3. Wstępnie oblicza mapę kontaktów (które koraliki są sąsiadami)
protein, interaction, contact_map, distance_map = setup_folding_system(
    main_chain=main_chain, 
    side_chain=side_chain
)

🧬 Analiza sekwencji: APRLRFY


### 🔨 2. Konstrukcja Hamiltonianu

**Hamiltonian** ($H$) jest sercem symulacji. Reprezentuje całkowitą energię układu jako sumę składników:

$$ H_{total} = H_{interaction} + H_{backbone} + H_{chirality} + H_{overlap} $$

- **$H_{interaction}$**: Energia ujemna (dobrze), gdy stykają się odpowiednie aminokwasy.
- **$H_{backbone}$**: Duża dodatnia kara (źle), jeśli łańcuch zostanie przerwany (szkielet).
- **$H_{chirality}$**: Kara, jeśli łańcuch wykonuje niemożliwe skręty (chiralność).
- **$H_{overlap}$**: Kara, jeśli dwa węzły znajdują się w tym samym miejscu (nakładanie).

**Kompresja:** Uruchamiamy również krok kompresji, aby usunąć człony Pauliego, które są tożsamościowe (all-identity).

In [11]:
from utils.setup_utils import build_and_compress_hamiltonian

original_h, compressed_h = build_and_compress_hamiltonian(
    protein=protein,
    interaction=interaction,
    contact_map=contact_map,
    distance_map=distance_map,
)

print(f"📉 Zredukowano przestrzeń optymalizacji: {original_h.num_qubits} kubitów -> {compressed_h.num_qubits} kubitów")

📉 Zredukowano przestrzeń optymalizacji: 48 kubitów -> 9 kubitów


### 🚀 3. Rozwiązanie za pomocą VQE

Używamy **VQE (Variational Quantum Eigensolver)**, aby znaleźć konfigurację minimalizującą Hamiltonian.

**Jak to działa:**
1. Komputer kwantowy (Ansatz) przygotowuje stan próbny.
2. Mierzymy energię tego stanu.
3. Optymalizator klasyczny (COBYLA) dostraja parametry, aby obniżyć energię.
4. Powtarzamy do momentu zbieżności (konwergencji).

In [12]:
from utils.setup_utils import setup_vqe_optimization, run_vqe_optimization, setup_result_analysis

# 1. Konfiguracja VQE (Ansatz + Optymalizator)
vqe, counts, values = setup_vqe_optimization(num_qubits=compressed_h.num_qubits)

# 2. Uruchomienie optymalizacji
print("⏳ VQE pracuje... (To może chwilę potrwać)")
raw_results = run_vqe_optimization(vqe=vqe, hamiltonian=compressed_h)

print(f"✅ Optymalizacja zakończona!")
print(f"   Znaleziona energia minimalna: {raw_results.eigenvalue.real:.4f}")

⏳ VQE pracuje... (To może chwilę potrwać)
✅ Optymalizacja zakończona!
   Znaleziona energia minimalna: -1.3963


### 📊 4. Analiza i wizualizacja

Na koniec dekodujemy wynik kwantowy. VQE zwraca ciąg bitów reprezentujący sekwencję **skrętów** w sieci (np. "Prawo, Góra, Lewo"). Konwertujemy tę sekwencję skrętów na współrzędne kartezjańskie (X, Y, Z) i wykreślamy ją.

In [13]:
print("Interpretacja i wizualizacja wyników... (to może chwilę potrwać)")

# Interpretacja wyników
result_interpreter, result_visualizer = setup_result_analysis(
    raw_results=raw_results,
    protein=protein,
    vqe_iterations=counts,
    vqe_energies=values,
)

# Zapisz interpretację wyników do plików
result_interpreter.dump_results_to_files()

# Generuj wizualizacje 3D
result_visualizer.visualize_3d()
result_visualizer.visualize_2d()
result_visualizer.generate_3d_gif()

print(f"Jeśli chcesz zobaczyć pliki i wizualizacje dla tego konkretnego uruchomienia - sprawdź: {result_visualizer._dirpath}")


Interpretacja i wizualizacja wyników... (to może chwilę potrwać)
Jeśli chcesz zobaczyć pliki i wizualizacje dla tego konkretnego uruchomienia - sprawdź: ...\quantum-protein-folding\output\results\XXXX_XX_XX-XX_XX_XX-APRLRFY-_______


Jeśli chcesz zobaczyć, jak poradziliśmy sobie z wizualizacją danych 3D, możesz sprawdzić:
- [Interaktywny wykres 3D](https://qfold-thesis.github.io/quantum-protein-folding/assets/interactive_3d_visualization.html)
- [GIF obracającej się konformacji 3D](https://raw.githubusercontent.com/QFold-Thesis/quantum-protein-folding/gh-pages/assets/rotating_3d_visualization.gif)


---
# 🔴 Część 2: Ręczny przepływ pracy

Jeśli chcesz zmodyfikować konkretne parametry (takie jak zmiana **Modelu Oddziaływań** z MJ na HP lub dostrojenie **Wartości Kar**), powinieneś skorzystać z przepływu ręcznego.

### Krok 1: Wybierz model oddziaływań

Odkomentuj poniżej model, którego chcesz użyć. Pamiętaj, że sekwencja musi być zgodna z oczekiwanymi symbolami modelu.

In [14]:
from protein import Protein
from interaction import HPInteraction, MJInteraction
from constants import EMPTY_SIDECHAIN_PLACEHOLDER

# --- OPCJA A: Model MJ (Miyazawa-Jernigan) ---
manual_interaction = MJInteraction()
seq = "APRLRFY"

# --- OPCJA B: Model HP (Hydrofobowo-Polarny) ---
# manual_interaction = HPInteraction()

print(f"Używany model: {manual_interaction.__class__.__name__}")
print(f"Prawidłowe symbole: {manual_interaction.valid_symbols}")

# Inicjalizacja białka
manual_protein = Protein(
    main_protein_sequence=seq,
    side_protein_sequence=EMPTY_SIDECHAIN_PLACEHOLDER * len(seq),
    valid_symbols=manual_interaction.valid_symbols
)

Używany model: MJInteraction
Prawidłowe symbole: {'P', 'H', 'S', 'C', 'D', 'F', 'L', 'Y', 'G', 'I', 'V', 'W', 'Q', 'E', 'K', 'R', 'T', 'A', 'N', 'M'}


### Krok 2: Konstrukcja Hamiltonianu (Ręcznie)

Tutaj ręcznie tworzymy instancje `ContactMap` i `DistanceMap`. Są to kosztowne obliczeniowo wstępne kalkulacje, które określają prawidłowe ścieżki skręcania.

In [15]:
from contact import ContactMap
from distance import DistanceMap
from builder import HamiltonianBuilder
from utils.qubit_utils import remove_unused_qubits

manual_contact_map = ContactMap(manual_protein)
manual_distance_map = DistanceMap(manual_protein)

h_builder = HamiltonianBuilder(
    protein=manual_protein,
    interaction=manual_interaction,
    distance_map=manual_distance_map,
    contact_map=manual_contact_map,
)

full_h = h_builder.sum_hamiltonians()
compressed_h = remove_unused_qubits(full_h)

print(f"Ręczna konstrukcja zakończona. Rozmiar Hamiltonianu: {compressed_h.num_qubits} kubitów")

Ręczna konstrukcja zakończona. Rozmiar Hamiltonianu: 9 kubitów


### Krok 3: Niestandardowe rozwiązanie

Możesz teraz podłączyć ten Hamiltonian do dowolnego solvera Qiskit (Sampler, Estimator, VQE, QAOA).

In [16]:
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import real_amplitudes
from backend import get_sampler

# Konfiguracja Ansatzu
ansatz = real_amplitudes(num_qubits=compressed_h.num_qubits, reps=1)
# Konfiguracja optymalizatora
optimizer = COBYLA(maxiter=50)

# Pobierz Sampler (Backend)
sampler, _ = get_sampler()

# Inicjalizacja VQE
vqe = SamplingVQE(
    sampler=sampler,
    ansatz=ansatz,
    optimizer=optimizer,
    aggregation=0.1
)

# Uruchomienie
result = vqe.compute_minimum_eigenvalue(compressed_h)
print(f"Ręczny wynik VQE: {result.eigenvalue}")

Ręczny wynik VQE: -1.398548828124523
